RNN 으로 영화 리뷰 감정 분석하기
RNN 을 이용하여 IMDB 의 영화 리뷰가 긍정적인지 부정적인지 구분해봅시다.

In [1]:
import os
import sys
import datetime
import argparse
import torch
from torchtext import data, datasets
from torch import autograd
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
import copy
import spacy

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

HyperParameter 정의하기

In [3]:
# get hyper parameters
lr = 0.0003
epochs = 8
batch_size = 32

In [4]:
print("\nLoading data...")
TEXT = data.Field(sequential=True,  lower=True)
LABEL = data.Field(sequential=False)
train, test = datasets.IMDB.splits(TEXT, LABEL)
TEXT.build_vocab(train, min_freq=5)
LABEL.build_vocab(train)

train_iter, test_iter = data.BucketIterator.splits(
    (train, test), batch_size=32, shuffle=False, repeat=False, device=-1)




vocab_size = len(TEXT.vocab)
n_classes = len(LABEL.vocab) - 1


Loading data...


IMDB 데이터셋 불러오기

Tockenizing 이 끝난 데이터 하나를 살펴보겠습니다. list of words 로 표현되는 데이터가 positive 혹은 negative 레이블을 띄고 있는것이 확인 가능합니다.

In [159]:
b, batch = (next(enumerate(train_iter)))

# for i in range(0, batch_size):
review = batch.text
sentiment = batch.label
# for i in range(0, batch_size):
print([TEXT.vocab.itos[j] for j in review.data[:,0]] )
print(sentiment.data[0])
#1 is neg, 2 is pos

['i', 'loved', 'this', 'movie.', 'i', 'totally', 'disagree', 'with', 'some', '<unk>', 'critiques', 'that', "i've", 'read', 'over', 'the', 'years.', 'this', 'was', 'a', 'great', 'vehicle', 'for', 'eddie', '<unk>', 'he', 'appeared', 'to', 'have', 'a', 'great', 'time', 'with', 'his', 'part', 'as', 'chandler', '<unk>', 'and', 'he', 'should', 'never', 'care', 'about', 'what', 'the', 'critics', 'say,', 'if', 'he', 'had', 'fun', 'doing', 'it', '\x96', 'and', 'most', 'of', 'his', 'audience', 'enjoyed', 'it!', 'and,', 'it', 'was', 'fun', 'to', 'watch', 'as', 'it', 'combined', 'some', 'great', 'fantasy', 'tension', 'with', 'mr.', "murphy's", 'great', 'comedic', 'style.', 'you', 'have', 'to', 'keep', 'in', 'mind', 'that', '<unk>', "child'", 'is', 'a', '<unk>', 'film', '\x96', 'just', 'an', 'imaginative', 'work', 'of', 'magic', 'and', 'wonder', 'amidst', 'the', "'real'", 'world.', 'during', 'the', 'time', 'this', 'film', 'was', 'released,', 'i', 'was', 'working', 'in', 'a', 'video', 'rental', 'sto

RNN Classifier 객체 정의하기

In [160]:
class BasicRNN(nn.Module):
    def __init__(self, batch_size, vocab_size, embed_dim, hidden_dim, n_classes, dropout=0.2,  n_layers=1):
        super(BasicRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, n_layers)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, source_sentence):
        embedded = self.embedding(source_sentence)
        init_h = self.init_hidden(self.batch_size)        
        _, hidden = self.gru(embedded, init_h)
        self.dropout(hidden)
        hidden = hidden.squeeze(0)
        linear = self.linear(hidden)
        return linear

    def init_hidden(self, batch_size):
        hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim) )
        return hidden

한 Batch 를 학습하는 함수 구현하기

In [183]:
def train(model, optimizer, train_iter):
    """Train Model"""
    #모델을 train 모드로 바꿔줌
#     criterion = nn.BCEWithLogitsLoss()
    model.train()
    for b, batch in enumerate(train_iter):
        #batch 속의 데이터와 레이블을 전에 정한 기기로 보내줍니다. 
        x, y = batch.text.to(device), batch.label.to(device)

        #레이블을 1,2 에서 1을 빼 0,1 값으로 바꿔줍니다.
        y.data.sub_(1)
        optimizer.zero_grad()

        x = Variable(x)
        y = Variable(y)
        
        logit = model(x).squeeze()

        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        print("loss at batch ",b, " is ", loss.item() )# %d: %5.2f",%(b, loss.item()) )
#         batch 속의 100번째 데이터 마다 오차와 정확도를 출력합니다.
        if b % 10 == 0:
#             print("Training Loss at ", b, ": ", loss)
            corrects = (torch.max(logit, 1)[1].view(y.size()).data == y.data).sum()
            accuracy = 100.0 * corrects / batch.batch_size
            sys.stdout.write(
                '\rBatch[%d] - loss: %.6f  acc: %.4f (%d/%d)' % (
                b, loss.item(), accuracy, corrects, batch.batch_size))

모델 성능을 평가하는 함수 구현

In [182]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, avg_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        

        
        logit = None
        try:
            logit = model(x)
        except:
            continue
        loss = F.cross_entropy(logit, y, size_average=False)
        avg_loss += loss.data[0]
        corrects += (torch.max(logit, 1)
                     [1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = avg_loss / size
    accuracy = 100.0 * corrects / size
    return avg_loss, accuracy

모델 객체를 생성하고 학습시킵니다.

In [163]:
model = BasicRNN(batch_size, vocab_size, 128, 256, n_classes) #(self, vocab_size, embed_dim, hidden_dim, n_classes, n_layers=1):
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
models = []
for e in range(1, 2):
    try:
        train(model, optimizer, train_iter)
        val_loss, val_accuracy = evaluate(model, val_iter)
    except RuntimeError:
        continue
        
    print("\n[Epoch: %d] val_loss:%5.2f | acc:%5.2f" % (e, val_loss, val_accuracy))
    models.append((e,val_loss, val_accuracy, copy.deepcopy(model)))

loss at batch  0  is  0.5652493834495544
Batch[0] - loss: 0.565249  acc: 100.0000 (32/32)loss at batch  1  is  0.45410099625587463
loss at batch  2  is  0.5111140608787537
loss at batch  3  is  0.21587389707565308
loss at batch  4  is  0.21787601709365845
loss at batch  5  is  0.3344394862651825
loss at batch  6  is  0.21996352076530457
loss at batch  7  is  0.0859660655260086
loss at batch  8  is  0.26749616861343384
loss at batch  9  is  0.09539347141981125
loss at batch  10  is  0.08991160988807678
Batch[10] - loss: 0.089912  acc: 100.0000 (32/32)loss at batch  11  is  0.03961355239152908
loss at batch  12  is  0.11098809540271759
loss at batch  13  is  0.13442201912403107
loss at batch  14  is  0.0903748944401741
loss at batch  15  is  0.056058939546346664
loss at batch  16  is  0.030013248324394226
loss at batch  17  is  0.10722120106220245
loss at batch  18  is  0.10355749726295471
loss at batch  19  is  0.06859183311462402
loss at batch  20  is  0.023878920823335648
Batch[20] - 

loss at batch  164  is  0.00010323867172701284
loss at batch  165  is  2.1082862076582387e-05
loss at batch  166  is  3.083118281210773e-05
loss at batch  167  is  1.857077222666703e-05
loss at batch  168  is  2.5206238206010312e-05
loss at batch  169  is  2.9013359380769543e-05
loss at batch  170  is  7.928466948214918e-05
Batch[170] - loss: 0.000079  acc: 100.0000 (32/32)loss at batch  171  is  3.264181214035489e-05
loss at batch  172  is  6.001932706567459e-05
loss at batch  173  is  5.8356494264444336e-05
loss at batch  174  is  7.768708019284531e-05
loss at batch  175  is  2.140266224159859e-05
loss at batch  176  is  0.00010879071487579495
loss at batch  177  is  6.352380296448246e-05
loss at batch  178  is  2.308665352757089e-05
loss at batch  179  is  9.744651470100507e-05
loss at batch  180  is  3.678338907775469e-05
Batch[180] - loss: 0.000037  acc: 100.0000 (32/32)loss at batch  181  is  3.9344067772617564e-05
loss at batch  182  is  1.8176735466113314e-05
loss at batch  183

loss at batch  325  is  0.26102307438850403
loss at batch  326  is  0.3584638237953186
loss at batch  327  is  0.18770639598369598
loss at batch  328  is  0.34312793612480164
loss at batch  329  is  0.30500680208206177
loss at batch  330  is  0.18869107961654663
Batch[330] - loss: 0.188691  acc: 100.0000 (32/32)loss at batch  331  is  0.4990672171115875
loss at batch  332  is  0.1537095159292221
loss at batch  333  is  0.20676341652870178
loss at batch  334  is  0.14256471395492554
loss at batch  335  is  0.18512524664402008
loss at batch  336  is  0.2887139618396759
loss at batch  337  is  0.06311166286468506
loss at batch  338  is  0.6765438914299011
loss at batch  339  is  0.8747435808181763
loss at batch  340  is  0.1444452702999115
Batch[340] - loss: 0.144445  acc: 96.0000 (31/32)loss at batch  341  is  0.25540226697921753
loss at batch  342  is  0.41495150327682495
loss at batch  343  is  0.0678262859582901
loss at batch  344  is  0.5676963925361633
loss at batch  345  is  0.4297

Batch[650] - loss: 0.000505  acc: 100.0000 (32/32)loss at batch  651  is  0.00030421462724916637
loss at batch  652  is  0.0003417448897380382
loss at batch  653  is  0.0006185882375575602
loss at batch  654  is  0.0002499150868970901
loss at batch  655  is  0.0002828915312420577
loss at batch  656  is  0.00038316898280754685
loss at batch  657  is  0.00028545211534947157
loss at batch  658  is  0.0002749522100202739
loss at batch  659  is  0.0004770187952090055
loss at batch  660  is  0.0002362413506489247
Batch[660] - loss: 0.000236  acc: 100.0000 (32/32)loss at batch  661  is  0.0008134146919474006
loss at batch  662  is  0.0004458077601157129
loss at batch  663  is  0.00044291664380580187
loss at batch  664  is  0.0005767394322901964
loss at batch  665  is  0.00035673039383254945
loss at batch  666  is  0.00028237298829481006
loss at batch  667  is  0.0006395908421836793
loss at batch  668  is  0.000587812508456409
loss at batch  669  is  0.0003067806246690452
loss at batch  670  i

Validation Accuracy 가 가장 큰 모델 저장하기

In [181]:

val_loss, val_accuracy = evaluate(model, test_iter)

/Users/sangjunyum/anaconda/lib/python3.6/site-packages/torchtext-0.2.0-py3.6.egg/torchtext/data/field.py:310: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/Users/sangjunyum/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor([[    86,     88,     15,     10,     49,      2,      3,     10,
             10,     10,     46,    794,   3783,   5862,     16,      3,
             30,    191,     46,     10,   1780,    371,    760,     39,
              3,     10,    627,     48,     51,      9,    308,   4204],
        [   395,    448,      3,      7,   1246,  29859,     88,     20,
              7,      7,    788,    278,   4285,   6488,   1016,   1468,
              5,      3,     25,      7,     12,  20278,     10,    120,
           8795,      7,      4,      3,  10021,    418,      2,      0],
        [   111,     88,    196,      2,   1503,   1879,     24,      7,
              3,    168,    125,      5,    360,      3,   3847,    117,
              2,   5053,     37,      3,     17,    965,    105,      2,
             24,      3,    867,   1315,     51,     10,   1248,      0],
        [    84,   2019,    376,    745,  13388,     17,      8,   1828,
            495,    807,     62,     89,     17,

tensor([[   190,     87,     12,  ...,      9,     10,     48],
        [   119,      4,     14,  ...,     98,     20,      3],
        [ 24352,      3,   1363,  ...,     11,     62,    917],
        ...,
        [ 24352,     10,      3,  ...,   1195,     11,    978],
        [  3326,     20,     56,  ...,      0,   7102,  10320],
        [  2661,   3922,  34402,  ...,      1,      1,      1]]) tensor([ 0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0])
tensor([[     9,     10,    108,  ...,   2135,      2,      2],
        [   113,    397,     13,  ...,      5,    976,   1121],
        [    23,    191,    540,  ...,   1304,      5,    321],
        ...,
        [  1448,    463,     84,  ...,     25,    264,      8],
        [    49,   2717,   3552,  ...,  14477,    598,      0],
        [     0,   1045,   3552,  ...,      1,      1,      1]]) tensor([ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,

tensor([[     9,    485,      2,  ...,      2,     52,     44],
        [   680,      3,    211,  ...,   7504,      7,   1896],
        [    16,     49,     20,  ...,    731,     60,     18],
        ...,
        [   965,     12,     52,  ...,    242,   8399,     64],
        [    12,    166,    262,  ...,      2,     57,     28],
        [   699,    491,    457,  ...,    117,   5764,      0]]) tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1])
tensor([[     9,   4237,    108,  ...,      9,     49,     33],
        [    39,   5449,     13,  ...,    113,      0,   1151],
        [  1978,      7,  17368,  ...,     23,    168,     65],
        ...,
        [   300,    305,      5,  ...,      0,    503,     60],
        [     6,    207,      2,  ...,  13266,      0,  15601],
        [   922,  11576,  16670,  ...,      1,      1,      1]]) tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,

tensor([[     9,     10,      9,  ...,      2,     10,      9],
        [   986,    797,   1008,  ...,     78,     20,     86],
        [    12,      7,      9,  ...,   4513,    297,     98],
        ...,
        [    43,     67,    188,  ...,     70,    197,     28],
        [    44,     10,   1555,  ...,      0,   6457,      0],
        [ 25055,      0,  10673,  ...,      1,      1,      1]]) tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0])
tensor([[    10,     10,     10,  ...,      9,     10,     10],
        [    20,     24,    134,  ...,     82,     82,      7],
        [     7,      7,     14,  ...,    200,     84,     30],
        ...,
        [     3,      0,   1961,  ...,   1201,    154,      8],
        [   207,     13,    266,  ...,      0,   3196,  33134],
        [ 11576,    711,    664,  ...,      1,      1,      1]]) tensor([ 1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,

...

tensor([[ 14590,      0,     16,  ...,    980,      9,      0],
        [   564,    291,      3,  ...,     73,    365,  38881],
        [   103,    168,    904,  ...,     10,    296,   7579],
        ...,
        [    13,      6,   4499,  ...,      1,      1,      1],
        [  8278,    161,      4,  ...,      1,      1,      1],
        [     0,    664,   2240,  ...,      1,      1,      1]]) tensor([ 1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,
         1,  1,  1,  1])
tensor([[    10,    385,     44,  ...,     10,      9,      0],
        [   252,     74,   2450,  ...,      7,   1520,  22271],
        [     7,    596,     11,  ...,      3,     10,    108],
        ...,
        [    13,      4,      8,  ...,      1,      1,      1],
        [   742,     91,     92,  ...,      1,      1,      1],
        [  7152,  14639,    139,  ...,      1,      1,      1]]) tensor([ 0,  0,  0,  0,  0,  1,  1,  0,  1,  1,  1,

tensor([[     0,      9,   1264,  ...,    102,   5011,      0],
        [     8,    698,    980,  ...,     71,   3728,      7],
        [     0,    563,    623,  ...,  17745,     41,     32],
        ...,
        [     5,    509,      8,  ...,      1,      1,      1],
        [    35,    335,      2,  ...,      1,      1,      1],
        [   293,    112,   2440,  ...,      1,      1,      1]]) tensor([ 0,  0,  1,  1,  1,  1,  1,  1,  0,  1,  1,  0,  0,  1,
         1,  1,  0,  0,  0,  1,  1,  1,  1,  0,  0,  1,  1,  0,
         0,  1,  1,  1])
tensor([[    40,    281,  29235,  ...,  35941,  24435,      0],
        [   288,     83,      2,  ...,      0,  10441,   2405],
        [  1230,    131,    225,  ...,    284,   1498,      0],
        ...,
        [    40,      0,      0,  ...,      1,      1,      1],
        [   288,   5313,      1,  ...,      1,      1,      1],
        [  1230,      1,      1,  ...,      1,      1,      1]]) tensor([ 1,  0,  0,  1,  1,  0,  0,  0,  0,  1,  1,

tensor([[    12,     55,  14340,  ...,   1952,   4749,     30],
        [   141,     17,     14,  ...,     25,      8,      5],
        [   783,     35,      3,  ...,   9923,     63,      2],
        ...,
        [     0,    190,   5400,  ...,      1,      1,      1],
        [    13,    125,  21082,  ...,      1,      1,      1],
        [     0,    705,      1,  ...,      1,      1,      1]]) tensor([ 1,  1,  0,  0,  1,  0,  0,  1,  1,  1,  0,  0,  1,  1,
         1,  0,  0,  0,  1,  1,  0,  1,  1,  1,  1,  1,  1,  1,
         1,  0,  1,  1])
tensor([[  1952,      8,      0,  ...,      3,    501,      3],
        [   224,    652,      0,  ...,   2037,     50,    185],
        [  1947,      6,      0,  ...,   2911,      9,    921],
        ...,
        [     3,      7,    229,  ...,      1,      1,      1],
        [  7811,     23,     84,  ...,      1,      1,      1],
        [ 11265,    607,   9455,  ...,      1,      1,      1]]) tensor([ 0,  0,  1,  1,  1,  1,  1,  0,  0,  1,  0,

tensor([[     9,     30,      2,  ...,     35,      0,     15],
        [   200,     41,  27718,  ...,      9,     30,      9],
        [    10,      6,    160,  ...,     90,      5,    986],
        ...,
        [   805,      6,    222,  ...,      1,      1,      1],
        [    44,     28,  27718,  ...,      1,      1,      1],
        [  4917,      0,  18573,  ...,      1,      1,      1]]) tensor([ 0,  1,  1,  1,  0,  1,  1,  1,  1,  0,  0,  0,  0,  1,
         0,  0,  0,  1,  1,  1,  1,  1,  0,  0,  1,  0,  0,  1,
         1,  0,  1,  1])
tensor([[    57,   9252,      9,  ...,    144,    427,     10],
        [ 44605,     14,   4202,  ...,     54,     19,     14],
        [    16,    222,     11,  ...,   3284,      2,     30],
        ...,
        [    13,      5,     83,  ...,      1,      1,      1],
        [     0,      2,   9370,  ...,      1,      1,      1],
        [ 31340,      0,      0,  ...,      1,      1,      1]]) tensor([ 0,  1,  1,  1,  1,  0,  0,  0,  0,  1,  1,

/Users/sangjunyum/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


In [179]:
print(val_loss, val_accuracy)

tensor(3.6658) tensor(49)


In [ ]:
epoch_order = 0
best_val_accuracy = -1
best_val_loss = 0

for copy_model in models:
    if copy_model[2]> best_val_accuracy:
        epoch_order = copy_model[0]
        best_val_loss = copy_model[1]
        best_val_accuracy = copy_model[2]
        model = copy_model[3]

In [240]:
print("[!] saving model")
print("Best Validation Loss: %5.2f | Best Validation Accuracy: %5.2f", %())
if not os.path.isdir("snapshot"):
    os.makedirs("snapshot")
    torch.save(model.state_dict(), './snapshot/%d_convcnn.pt' % (epoch_order))

SyntaxError: invalid syntax (<ipython-input-240-b7950cb47c7b>, line 2)

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
print(input.shape)
print(target.shape)

In [27]:
print(vocab_size)

46159
